# Prepare data

Load the train data and compute the area of defects

In [12]:
import pandas as pd
import numpy as np
from fastai.vision.all import *


path = Path("/data/datasets/severstal")
df = pd.read_csv(path/'train.csv')

df['DefectArea']=df['EncodedPixels'].apply(lambda ep: np.array(ep.split()[1::2]).astype('int').sum())
df.head()

,ImageId,ClassId,EncodedPixels,DefectArea
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 30370 24 30626 24 30882 24 31139 23 31395 23 31651 23 31907 23 32163 23 32419 23 32675 23 77918 27 78174 55 78429 60 78685 64 78941 68 79197 72 79452 77 79708 81 79964 85 80220 89 80475 94 80731 98 80987 102 81242 105 81498 105 81754 104 82010 104 82265 105 82521 31 82556 69 82779 27 82818 63 83038 22 83080 57 83297 17 83342 50 83555 13 83604 44 83814 8 83866 37 84073 3 84128 31 84390 25 84652 18 84918 8 85239 10 85476 29 85714 47 85960 57 86216 57 86471 58 86727 58 86983 58 87238 59 87494 59 87750 59 88005 60 88261 60 88517 60 88772 61 89028 53...,4396
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 110 19859 110 20115 110 20371 110 20627 110 20883 110 21139 110 21395 110 21651 110 21962 55 293125 251 293381 251 293637 251 293893 251 294149 251 294405 251 294661 251 294917 251 295173 251 295429 251 295685 251 295941 251 296197 251 296453 251 296709 251 296965 251 297221 251 297477 251 297733 251 297989 251 298245 251 298564 188 298945 63,6897
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 38863 28 39119 28 39375 29 39631 29 39887 29 40143 29 40399 29 40655 30 40911 30 41167 30 41423 30 41679 31 41935 31 42191 31 42447 31 42703 31 42960 31 43216 31 43472 31 43728 31 43984 31 44240 32 44496 32 44752 32 45008 32 45264 33 45520 33 45776 33 46032 33 46288 33 46544 34 46803 31 47065 25 47327 19 47588 15 47850 9 48112 3 62667 12 62923 23 63179 23 63348 3 63435 23 63604 7 63691 23 63860 11 63947 23 64116 15 64203 23 64372 19 64459 23 64628 24 64715 23 64884 28 64971 23 65139 33 65227 23 65395 37 65483 23 65651 41 65740 22 65907 45 65996 22...,8319
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 133248 13 133503 16 133757 19 134012 22 134267 24 134522 26 134777 29 135032 31 135287 34 135542 36 135796 40 136050 43 136304 46 136558 50 136812 54 137066 56 137320 59 137574 61 137828 63 138082 65 138336 68 138590 70 138845 71 139101 71 139356 73 139612 73 139868 73 140123 74 140379 74 140634 75 140890 75 141145 77 141400 78 141654 80 141909 81 142164 82 142418 84 142673 85 142928 86 143182 88 143437 89 143692 90 143946 93 144201 94 144456 95 144710 97 144965 98 145220 99 145474 101 145729 103 145983 105 146237 107 146491 109 146745 112 1469...,69357
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468 92 230623 10 230724 95 230845 11 230869 28 230979 97 231094 63 231235 97 231344 70 231490 99 231593 79 231746 183 232002 184 232257 186 232513 186 232769 185 233024 186 233280 185 233535 186 233791 185 234047 185 234302 186 234558 186 234813 186 235069 186 235327 184 235586 181 235845 177 236103 175 236362 172 236621 78 236714 71 236880 57 236972 61 237139 44 237229 52 237397 32 237486 11 237513 16 237656 19 237744 3 237915 6,4851


# Distribution of defects

In [25]:
summary = df.groupby(['ImageId','ClassId']).agg({'DefectArea':['count','sum']}).unstack('ClassId').fillna(0).astype(int)
summary.columns = ['Divots','Cracks','Scratches','Ablations', 'DivotsArea','CracksArea','ScratchesArea','AblationsArea']
summary['TotalDefects'] = summary.loc[:,'Divots':'Ablations'].sum(axis=1)
summary['TotalArea'] = summary.loc[:,'DivotsArea':'AblationsArea'].sum(axis=1)
summary

,Divots,Cracks,Scratches,Ablations,DivotsArea,CracksArea,ScratchesArea,AblationsArea,TotalDefects,TotalArea
ImageId,,,,,,,,,,
0002cc93b.jpg,1,0,0,0,4396,0,0,0,1,4396
0007a71bf.jpg,0,0,1,0,0,0,6897,0,1,6897
000a4bcdd.jpg,1,0,0,0,8319,0,0,0,1,8319
000f6bf48.jpg,0,0,0,1,0,0,0,69357,1,69357
0014fce06.jpg,0,0,1,0,0,0,4851,0,1,4851
...,...,...,...,...,...,...,...,...,...,...
ffcf72ecf.jpg,0,0,1,0,0,0,8903,0,1,8903
fff02e9c5.jpg,0,0,1,0,0,0,7206,0,1,7206
fffe98443.jpg,0,0,1,0,0,0,2336,0,1,2336


In [27]:
summary['TotalDefects'].value_counts()

1    6239
2     425
3       2
Name: TotalDefects, dtype: int64

In [28]:
summary.sum()

Divots                 897
Cracks                 247
Scratches             5150
Ablations              801
DivotsArea         3912129
CracksArea          834471
ScratchesArea    131306899
AblationsArea     27533572
TotalDefects          7095
TotalArea        163587071
dtype: int64